In [145]:
# Data wrangling
import pandas as pd
import numpy as np
import re

In [146]:
# Machine learning

import xgboost as xgb
from sklearn.model_selection import train_test_split

In [147]:
# Hyperparameters tuning

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [148]:
# Some constants

SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'Elapsed'

In [149]:
train_df = pd.read_csv('data/train_w_areas_st_till_june.csv',index_col=0)

### FIx: число предсказаний не совпадает с числом подаваемых данных
# train_df.dropna(inplace=True)

train_df.fillna(0, inplace=True)

train_df.drop(columns=['Start', 'JobName'], inplace=True)
train_df

,UID,GID,Area,Partition,ReqNodes,ReqCPUS,Timelimit,Submit,Elapsed,Priority,State,ExitCode
0,1420,88068,astrophys,tornado,70,1960,10-00:00:00,2021-08-11T16:53:04,1-11:27:29,1,CANCELLED by 50260,0:0
1,52951,3414,phys,tornado,2,56,14-00:00:00,2021-08-18T10:38:10,14-00:00:38,1,TIMEOUT,0:0
2,24801,97723,mech,tornado,6,168,14-00:00:00,2021-08-18T14:50:05,13-21:05:40,1,CANCELLED by 50341,0:0
3,24801,97723,mech,tornado,6,168,14-00:00:00,2021-08-18T14:52:31,13-21:04:39,1,CANCELLED by 50341,0:0
4,70802,33028,energ,tornado-k40,1,1,14-00:00:00,2021-08-19T15:33:49,14-00:00:04,1,TIMEOUT,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
1290724,70637,47128,mech,tornado,1,28,10-00:00:00,2023-05-31T23:42:30,05:19:37,480,COMPLETED,0:0
1290725,70637,47128,mech,tornado,1,28,10-00:00:00,2023-05-31T23:42:47,05:25:22,480,COMPLETED,0:0
1290726,70637,47128,mech,tornado,1,28,10-00:00:00,2023-05-31T23:43:10,05:28:30,480,COMPLETED,0:0
1290727,72271,99243,phys,tornado,1,1,1-00:00:00,2023-05-31T23:51:20,00:01:06,233,COMPLETED,0:0


In [150]:
regexp_time_list = [
    r'(?P<days>.+?(?=-))-(?P<hours>.+?(?=:)):(?P<minutes>.+?(?=:)):(?P<seconds>\d+)',
    r'(?P<hours>.+?(?=:)):(?P<minutes>.+?(?=:)):(?P<seconds>\d+)'
]
compiled_regexps = [re.compile(regexp) for regexp in regexp_time_list]

def convert_time_to_seconds(element):
    for rcompile in compiled_regexps:
        rsearch = rcompile.search(element)
        if rsearch:
            try:
                return (int(rsearch.group('days')) * 24 + int(rsearch.group('hours'))) * 3600 + int(rsearch.group('minutes')) * 60 + int(rsearch.group('seconds'))
            except:
                return int(rsearch.group('hours')) * 3600 + int(rsearch.group('minutes')) * 60 + int(rsearch.group('seconds'))
            
train_df['Timelimit'] = train_df['Timelimit'].apply(convert_time_to_seconds)
train_df['Elapsed'] = train_df['Elapsed'].apply(convert_time_to_seconds)

In [151]:
train_df['Area'].replace({'geophys': 0,
 'radiophys': 1,
 'phys': 2,
 'bioinf': 4,
 'mach': 5,
 'biophys': 6,
 'it': 7,
 'mech': 8,
 'energ': 9,
 'astrophys': 10}, inplace=True)
 
train_df['Area'].astype(int)

0          10
1           2
2           8
3           8
4           9
           ..
1290724     8
1290725     8
1290726     8
1290727     2
1290728     2
Name: Area, Length: 1290729, dtype: int32

In [152]:
train_df['Partition'].replace({'tornado': 0,
 'g2': 1,
 'cascade': 2,
 'tornado-k40': 3,
 'nv': 4}, inplace=True)

train_df['Partition'].astype(int)

0          0
1          0
2          0
3          0
4          3
          ..
1290724    0
1290725    0
1290726    0
1290727    0
1290728    0
Name: Partition, Length: 1290729, dtype: int32

In [153]:
train_df['State'].replace({
    'COMPLETED': 0,
    'FAILED': 1,
    'TIMEOUT': 2,
    'NODE_FAIL': 3,
    'OUT_OF_MEMORY': 4
}, inplace=True)

train_df['State'].replace(r'(CANCELLED.+)|(CANCELLED)', 5, regex=True, inplace=True)

train_df['State'].astype(int)

0          5
1          2
2          5
3          5
4          2
          ..
1290724    0
1290725    0
1290726    0
1290727    0
1290728    0
Name: State, Length: 1290729, dtype: int32

In [154]:
train_df['ExitCode'].replace(r':','', regex=True, inplace=True)
train_df['ExitCode'] = train_df.ExitCode.astype(int)

train_df['ExitCode'].astype(int)

0          0
1          0
2          0
3          0
4          0
          ..
1290724    0
1290725    0
1290726    0
1290727    0
1290728    0
Name: ExitCode, Length: 1290729, dtype: int32

In [155]:
train_df.reset_index(inplace=True, drop=True)

In [156]:
train_df['Submit'] = pd.to_datetime(train_df['Submit'])
train_df['Month'] = train_df['Submit'].dt.month

# train_df.drop(columns=['Submit'], inplace=True)

In [157]:
train_df['Timelimit'].value_counts()

Timelimit
172800     985813
43200       99750
1209600     42363
86400       24690
864000      15954
            ...  
1166400         1
1500            1
24000           1
840             1
7800            1
Name: count, Length: 188, dtype: int64

### Добавление фичей

In [158]:
# Среднее время
mean_elapsed = pd.DataFrame(train_df[train_df['State'] == 0].groupby('UID').mean()['Elapsed'])
mean_elapsed.columns = ['mean_elapsed']
mean_elapsed.reset_index(inplace=True)
train_df = train_df.merge(mean_elapsed, on='UID', how='left')

In [159]:
# Количество успешных задач
succeded_task_count = pd.DataFrame(train_df[train_df['State'] == 0].groupby('UID').count()['State'])
succeded_task_count.columns = ['succeded_task_count']
succeded_task_count.reset_index(inplace=True)

In [160]:
# Количество неуспешно завершенных задач
failed_task_count = pd.DataFrame(train_df[train_df['State'] == 1].groupby('UID').count()['State'])
failed_task_count.columns = ['failed_task_count']
failed_task_count.reset_index(inplace=True)

In [161]:
# Количество задач с таймаутом
timeout_task_count = pd.DataFrame(train_df[train_df['State'] == 2].groupby('UID').count()['State'])
timeout_task_count.columns = ['timeout_task_count']
timeout_task_count.reset_index(inplace=True)

In [162]:
tasks = succeded_task_count.merge(failed_task_count, on='UID', how='left')
tasks = tasks.merge(timeout_task_count, on='UID', how='left')
tasks.fillna({'succeded_task_count': 0, 'failed_task_count': 0, 'timeout_task_count': 0}, inplace=True)

In [163]:
# Отношение успешно завершенных задач к таймаутам и ошибочным
tasks['succeded_task_proportion'] = tasks['succeded_task_count'] / (tasks['succeded_task_count'] + tasks['failed_task_count'] + tasks['timeout_task_count'])
train_df = train_df.merge(tasks, on='UID', how='left')

In [19]:
# Fix: использование апостериорно получаемых фичей

# train_df.drop(columns=['State'], inplace=True)
# train_df.drop(columns=['ExitCode'], inplace=True)

### Добавление фичей (оконная функция)

In [126]:
train_df['Submit'] = pd.to_datetime(train_df['Submit'])

In [127]:
train_df.sort_values(by=['UID', 'Submit'], inplace=True)

In [128]:
def calculate_features(group):
    group['mean_elapsed'] = group['Elapsed'].expanding(min_periods=1).mean()
    group['succeded_task_count'] = group['State'].expanding(min_periods=group.shape[0]//100).apply(lambda x: (x == 0).sum())
    group['failed_task_count'] = group['State'].expanding(min_periods=group.shape[0]//100).apply(lambda x: (x == 1).sum())
    group['timeout_task_count'] = group['State'].expanding(min_periods=group.shape[0]//100).apply(lambda x: (x == 2).sum())
    return group

In [129]:
train_df = train_df.groupby('UID').apply(calculate_features)

In [130]:
train_df

UID    GID  Area  Partition  ReqNodes  ReqCPUS  Timelimit  \
UID                                                                          
215   736429     215  76557     0          0         1        1     604800   
      736430     215  76557     0          0         1        1     604800   
      736431     215  76557     0          0         1        1     604800   
      736432     215  76557     0          0         1        1     604800   
      736433     215  76557     0          0         1        1     604800   
...              ...    ...   ...        ...       ...      ...        ...   
99914 1255605  99914  53299     8          0         1        1      86400   
      1255606  99914  53299     8          0         1        1      86400   
      1257038  99914  53299     8          0         1        1      86400   
      1257039  99914  53299     8          0         1        1      86400   
      1257209  99914  53299     8          0         1        1      86400   

                           Submit  Elapsed  Priority  State  ExitCode  Month  \
UID                                                                            
215   736429  2022-09-02 10:49:13        1      3838      0         0      9   
      736430  2022-09-02 10:49:13        1      3838      0         0      9   
      736431  2022-09-02 10:49:13        1      3838      0         0      9   
      736432  2022-09-02 10:49:13        2      3838      0         0      9   
      736433  2022-09-02 10:49:13        1      3838      0         0      9   
...                           ...      ...       ...    ...       ...    ...   
99914 1255605 2023-05-11 22:12:42    45050       831      0         0      5   
      1255606 2023-05-11 22:13:05    43757       831      0         0      5   
      1257038 2023-05-12 23:57:00    46613       681      0         0      5   
      1257039 2023-05-12 23:57:33    46508       681      0         0      5   
      1257209 2023-05-13 23:43:17    86426       667      2         0      5   

               mean_elapsed  succeded_task_count  failed_task_count  \
UID                                                                   
215   736429       1.000000                  1.0                0.0   
      736430       1.000000                  2.0                0.0   
      736431       1.000000                  3.0                0.0   
      736432       1.250000                  4.0                0.0   
      736433       1.200000                  5.0                0.0   
...                     ...                  ...                ...   
99914 1255605  22501.100000                  7.0                1.0   
      1255606  24433.454545                  8.0                1.0   
      1257038  26281.750000                  9.0                1.0   
      1257039  27837.615385                 10.0                1.0   
      1257209  32022.500000                 10.0                1.0   

               timeout_task_count  
UID                                
215   736429                  0.0  
      736430                  0.0  
      736431                  0.0  
      736432                  0.0  
      736433                  0.0  
...                           ...  
99914 1255605                 0.0  
      1255606                 0.0  
      1257038                 0.0  
      1257039                 0.0  
      1257209                 1.0  

[1290729 rows x 17 columns]

In [131]:
train_df.dtypes

UID                             int64
GID                             int64
Area                            int64
Partition                       int64
ReqNodes                        int64
ReqCPUS                         int64
Timelimit                       int64
Submit                 datetime64[ns]
Elapsed                         int64
Priority                        int64
State                           int64
ExitCode                        int32
Month                           int32
mean_elapsed                  float64
succeded_task_count           float64
failed_task_count             float64
timeout_task_count            float64
dtype: object

In [132]:
# Fix: использование апостериорно получаемых фичей

train_df.drop(columns=['State'], inplace=True)
train_df.drop(columns=['ExitCode'], inplace=True)

In [133]:
train_df.fillna(0, inplace=True)

In [134]:
train_df

UID    GID  Area  Partition  ReqNodes  ReqCPUS  Timelimit  \
UID                                                                          
215   736429     215  76557     0          0         1        1     604800   
      736430     215  76557     0          0         1        1     604800   
      736431     215  76557     0          0         1        1     604800   
      736432     215  76557     0          0         1        1     604800   
      736433     215  76557     0          0         1        1     604800   
...              ...    ...   ...        ...       ...      ...        ...   
99914 1255605  99914  53299     8          0         1        1      86400   
      1255606  99914  53299     8          0         1        1      86400   
      1257038  99914  53299     8          0         1        1      86400   
      1257039  99914  53299     8          0         1        1      86400   
      1257209  99914  53299     8          0         1        1      86400   

                           Submit  Elapsed  Priority  Month  mean_elapsed  \
UID                                                                         
215   736429  2022-09-02 10:49:13        1      3838      9      1.000000   
      736430  2022-09-02 10:49:13        1      3838      9      1.000000   
      736431  2022-09-02 10:49:13        1      3838      9      1.000000   
      736432  2022-09-02 10:49:13        2      3838      9      1.250000   
      736433  2022-09-02 10:49:13        1      3838      9      1.200000   
...                           ...      ...       ...    ...           ...   
99914 1255605 2023-05-11 22:12:42    45050       831      5  22501.100000   
      1255606 2023-05-11 22:13:05    43757       831      5  24433.454545   
      1257038 2023-05-12 23:57:00    46613       681      5  26281.750000   
      1257039 2023-05-12 23:57:33    46508       681      5  27837.615385   
      1257209 2023-05-13 23:43:17    86426       667      5  32022.500000   

               succeded_task_count  failed_task_count  timeout_task_count  
UID                                                                        
215   736429                   1.0                0.0                 0.0  
      736430                   2.0                0.0                 0.0  
      736431                   3.0                0.0                 0.0  
      736432                   4.0                0.0                 0.0  
      736433                   5.0                0.0                 0.0  
...                            ...                ...                 ...  
99914 1255605                  7.0                1.0                 0.0  
      1255606                  8.0                1.0                 0.0  
      1257038                  9.0                1.0                 0.0  
      1257039                 10.0                1.0                 0.0  
      1257209                 10.0                1.0                 1.0  

[1290729 rows x 15 columns]

### Случайное разбитие на трейн и тест

In [164]:
train_df.drop(columns=['Submit'], inplace=True)

In [170]:
train_df.drop(columns=['State','ExitCode'], inplace=True)

In [171]:
X = train_df.drop(columns=['Elapsed'])
y = train_df['Elapsed']

In [172]:
train_df['Elapsed']

0           127649
1          1209638
2          1199140
3          1199079
4          1209604
            ...   
1290724      19177
1290725      19522
1290726      19710
1290727         66
1290728      18114
Name: Elapsed, Length: 1290729, dtype: int64

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VALID_SIZE, random_state=SEED)

In [174]:
X_train.dtypes

UID                           int64
GID                           int64
Area                          int64
Partition                     int64
ReqNodes                      int64
ReqCPUS                       int64
Timelimit                     int64
Priority                      int64
Month                         int32
mean_elapsed                float64
succeded_task_count         float64
failed_task_count           float64
timeout_task_count          float64
succeded_task_proportion    float64
dtype: object

### Разбитие по дате

In [31]:
test_quantile = 0.8
test_date = train_df['Submit'].quantile(test_quantile)

test_date

Timestamp('2022-12-22 21:31:46')

In [32]:
train_df.sort_values(by=['Submit'], inplace=True)
train_df

UID    GID  Area  Partition  ReqNodes  ReqCPUS  Timelimit  \
UID                                                                          
1420  0         1420  88068    10          0        70     1960     864000   
52951 1        52951   3414     2          0         2       56    1209600   
24801 2        24801  97723     8          0         6      168    1209600   
      3        24801  97723     8          0         6      168    1209600   
70802 4        70802  33028     9          3         1        1    1209600   
...              ...    ...   ...        ...       ...      ...        ...   
70637 1290724  70637  47128     8          0         1       28     864000   
      1290725  70637  47128     8          0         1       28     864000   
      1290726  70637  47128     8          0         1       28     864000   
72271 1290727  72271  99243     2          0         1        1      86400   
      1290728  72271  99243     2          0         1        1      86400   

                           Submit  Elapsed  Priority  Month  mean_elapsed  \
UID                                                                         
1420  0       2021-08-11 16:53:04   127649         1      8      0.000000   
52951 1       2021-08-18 10:38:10  1209638         1      8      0.000000   
24801 2       2021-08-18 14:50:05  1199140         1      8      0.000000   
      3       2021-08-18 14:52:31  1199079         1      8      0.000000   
70802 4       2021-08-19 15:33:49  1209604         1      8      0.000000   
...                           ...      ...       ...    ...           ...   
70637 1290724 2023-05-31 23:42:30    19177       480      5  14731.261803   
      1290725 2023-05-31 23:42:47    19522       480      5  14733.317031   
      1290726 2023-05-31 23:43:10    19710       480      5  14735.451115   
72271 1290727 2023-05-31 23:51:20       66       233      5   1591.692308   
      1290728 2023-05-31 23:54:14    18114       233      5   2771.857143   

               succeded_task_count  failed_task_count  timeout_task_count  
UID                                                                        
1420  0                        0.0                0.0                 0.0  
52951 1                        0.0                0.0                 0.0  
24801 2                        0.0                0.0                 0.0  
      3                        0.0                0.0                 0.0  
70802 4                        0.0                0.0                 0.0  
...                            ...                ...                 ...  
70637 1290724               2085.0               93.0                 2.0  
      1290725               2086.0               93.0                 2.0  
      1290726               2087.0               93.0                 2.0  
72271 1290727                 12.0                0.0                 0.0  
      1290728                 13.0                0.0                 0.0  

[1290729 rows x 15 columns]

In [33]:
# Разбиваем датасет на train и test
train = train_df[train_df['Submit'] < test_date]
test = train_df[train_df['Submit'] >= test_date]

In [34]:
train

UID    GID  Area  Partition  ReqNodes  ReqCPUS  Timelimit  \
UID                                                                          
1420  0         1420  88068    10          0        70     1960     864000   
52951 1        52951   3414     2          0         2       56    1209600   
24801 2        24801  97723     8          0         6      168    1209600   
      3        24801  97723     8          0         6      168    1209600   
70802 4        70802  33028     9          3         1        1    1209600   
...              ...    ...   ...        ...       ...      ...        ...   
18860 1032461  18860  81411     4          0         1        1       7200   
      1032460  18860  81411     4          0         1        1       7200   
      1032459  18860  81411     4          0         1        1       7200   
      1032458  18860  81411     4          0         1        1       7200   
      1032463  18860  81411     4          0         1        1       7200   

                           Submit  Elapsed  Priority  Month  mean_elapsed  \
UID                                                                         
1420  0       2021-08-11 16:53:04   127649         1      8      0.000000   
52951 1       2021-08-18 10:38:10  1209638         1      8      0.000000   
24801 2       2021-08-18 14:50:05  1199140         1      8      0.000000   
      3       2021-08-18 14:52:31  1199079         1      8      0.000000   
70802 4       2021-08-19 15:33:49  1209604         1      8      0.000000   
...                           ...      ...       ...    ...           ...   
18860 1032461 2022-12-22 21:31:45     1808       466     12   3321.995627   
      1032460 2022-12-22 21:31:45     1404       466     12   3322.056930   
      1032459 2022-12-22 21:31:45      833       466     12   3322.134597   
      1032458 2022-12-22 21:31:45     1913       466     12   3322.235392   
      1032463 2022-12-22 21:31:45     1892       466     12   3321.869069   

               succeded_task_count  failed_task_count  timeout_task_count  
UID                                                                        
1420  0                        0.0                0.0                 0.0  
52951 1                        0.0                0.0                 0.0  
24801 2                        0.0                0.0                 0.0  
      3                        0.0                0.0                 0.0  
70802 4                        0.0                0.0                 0.0  
...                            ...                ...                 ...  
18860 1032461              23703.0              839.0                13.0  
      1032460              23702.0              839.0                13.0  
      1032459              23701.0              839.0                13.0  
      1032458              23700.0              839.0                13.0  
      1032463              23705.0              839.0                13.0  

[1032514 rows x 15 columns]

In [35]:
test

UID    GID  Area  Partition  ReqNodes  ReqCPUS  Timelimit  \
UID                                                                          
18860 1032570  18860  81411     4          0         1        1       7200   
      1032563  18860  81411     4          0         1        1       7200   
      1032569  18860  81411     4          0         1        1       7200   
      1032568  18860  81411     4          0         1        1       7200   
      1032567  18860  81411     4          0         1        1       7200   
...              ...    ...   ...        ...       ...      ...        ...   
70637 1290724  70637  47128     8          0         1       28     864000   
      1290725  70637  47128     8          0         1       28     864000   
      1290726  70637  47128     8          0         1       28     864000   
72271 1290727  72271  99243     2          0         1        1      86400   
      1290728  72271  99243     2          0         1        1      86400   

                           Submit  Elapsed  Priority  Month  mean_elapsed  \
UID                                                                         
18860 1032570 2022-12-22 21:31:46     1343       466     12   3316.407990   
      1032563 2022-12-22 21:31:46     1581       466     12   3316.846976   
      1032569 2022-12-22 21:31:46     1347       466     12   3316.487543   
      1032568 2022-12-22 21:31:46     1722       466     12   3316.566942   
      1032567 2022-12-22 21:31:46     1719       466     12   3316.631229   
...                           ...      ...       ...    ...           ...   
70637 1290724 2023-05-31 23:42:30    19177       480      5  14731.261803   
      1290725 2023-05-31 23:42:47    19522       480      5  14733.317031   
      1290726 2023-05-31 23:43:10    19710       480      5  14735.451115   
72271 1290727 2023-05-31 23:51:20       66       233      5   1591.692308   
      1290728 2023-05-31 23:54:14    18114       233      5   2771.857143   

               succeded_task_count  failed_task_count  timeout_task_count  
UID                                                                        
18860 1032570              23810.0              839.0                14.0  
      1032563              23803.0              839.0                14.0  
      1032569              23809.0              839.0                14.0  
      1032568              23808.0              839.0                14.0  
      1032567              23807.0              839.0                14.0  
...                            ...                ...                 ...  
70637 1290724               2085.0               93.0                 2.0  
      1290725               2086.0               93.0                 2.0  
      1290726               2087.0               93.0                 2.0  
72271 1290727                 12.0                0.0                 0.0  
      1290728                 13.0                0.0                 0.0  

[258215 rows x 15 columns]

In [36]:
X_train = train.drop(columns=['Elapsed','Submit'])
y_train = train['Elapsed']
X_test = test.drop(columns=['Elapsed','Submit'])
y_test = test['Elapsed']

In [37]:
X_train.dtypes

UID                      int64
GID                      int64
Area                     int64
Partition                int64
ReqNodes                 int64
ReqCPUS                  int64
Timelimit                int64
Priority                 int64
Month                    int32
mean_elapsed           float64
succeded_task_count    float64
failed_task_count      float64
timeout_task_count     float64
dtype: object

### Обучение

In [175]:
regressor=xgb.XGBRegressor(booster = 'gbtree',
                           colsample_bytree = 1.0,
                           learning_rate = 0.025, # xgb’s “eta”
                           gamma = 1.0,
                           max_depth = 13,
                           min_child_weight = 3,
                           n_estimators = 263,
                           n_jobs = 16,
                           objective = 'reg:squarederror',
                           random_state = SEED,
                           subsample = 0.7,
                           tree_method = 'exact'
                        )
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.025, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=13, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=263, n_jobs=16,
             num_parallel_tree=None, random_state=314159265, ...)

In [176]:
predictions = regressor.predict(X_test)

In [177]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, predictions)

print(f'R2 Score: {r2}')

R2 Score: 0.5777786516354317


In [178]:
import pickle

filename = 'SKPred_pickle_saved_model.sav'
pickle.dump(regressor, open(filename, 'wb'))

In [179]:
# Save with xgboost

regressor.save_model('SKPred_xgboost_saved_model.json')

# Hyperopt

In [36]:
def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    #watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    regressor = xgb.train(params, dtrain, num_round)
    predictions = regressor.predict(dvalid)
    score = r2_score(y_test, predictions)
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

In [1]:
def optimize(
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'objective': 'reg:squarederror',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 16,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest,  
                max_evals=250)
    return best

NameError: name 'SEED' is not defined

### Лучшие гиперпараметры
{'booster': 'gbtree', 'colsample_bytree': 1.0, 'eta': 0.025, 'gamma': 1.0, 'max_depth': 13, 'min_child_weight': 3.0, 'n_estimators': 263.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 0.7000000000000001, 'tree_method': 'exact'}

In [38]:
best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params:                                  
{'booster': 'gbtree', 'colsample_bytree': 0.9, 'eta': 0.17500000000000002, 'gamma': 0.8, 'max_depth': 7, 'min_child_weight': 4.0, 'n_estimators': 848.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 0.7000000000000001, 'tree_method': 'exact'}
  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

c:\Users\dimas\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:53:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



	Score 0.7172271439192176                              


Training with params:                                                                
{'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.275, 'gamma': 0.6000000000000001, 'max_depth': 10, 'min_child_weight': 3.0, 'n_estimators': 807.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
  0%|          | 1/250 [03:14<13:27:06, 194.48s/trial, best loss: 0.2827728560807824]

c:\Users\dimas\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:56:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



	Score 0.7061238695197123                                                            


Training with params:                                                                
{'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.05, 'gamma': 0.6000000000000001, 'max_depth': 2, 'min_child_weight': 6.0, 'n_estimators': 867.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 0.55, 'tree_method': 'exact'}
  1%|          | 2/250 [06:50<14:16:19, 207.18s/trial, best loss: 0.2827728560807824]

...

	Score 0.5477966575338711                                                             


Training with params:                                                                 
{'booster': 'gbtree', 'colsample_bytree': 0.65, 'eta': 0.07500000000000001, 'gamma': 0.65, 'max_depth': 9, 'min_child_weight': 2.0, 'n_estimators': 248.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
 99%|█████████▉| 248/250 [6:11:38<02:33, 76.73s/trial, best loss: 0.2700914375476715]

c:\Users\dimas\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [06:04:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



	Score 0.7215782436592991                                                            


Training with params:                                                                
{'booster': 'gbtree', 'colsample_bytree': 0.55, 'eta': 0.05, 'gamma': 0.7000000000000001, 'max_depth': 13, 'min_child_weight': 1.0, 'n_estimators': 664.0, 'nthread': 16, 'objective': 'reg:squarederror', 'seed': 314159265, 'silent': 1, 'subsample': 1.0, 'tree_method': 'exact'}
100%|█████████▉| 249/250 [6:12:28<01:08, 68.56s/trial, best loss: 0.2700914375476715]

c:\Users\dimas\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [06:05:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



	Score 0.722084441620104                                                             


100%|██████████| 250/250 [6:15:12<00:00, 90.05s/trial, best loss: 0.2700914375476715]
The best hyperparameters are:  

{'colsample_bytree': 1.0, 'eta': 0.025, 'gamma': 1.0, 'max_depth': 12, 'min_child_weight': 3.0, 'n_estimators': 263.0, 'subsample': 0.7000000000000001}
